# Assignment 7

Brandon Sams

17Oct2020

In this part of the assignment, you will partition a dataset using different strategies. You will use the routes.parquet dataset you created in a previous assignment. For this dataset, the key for each route will be the three-letter source airport code concatenated with the three-letter destination airport code and the two-letter airline. For instance, a route from Omaha Eppley Airfield (OMA) to Denver International Airport (DEN) on American Airlines (AA) has a key of OMADENAA.

## Part A: Instructions

Start by loading the dataset from the previous assignment using Pandas's read_parquet method. Next, add the concatenated key then using Panda's apply method to create a new column called key. For this part of the example, we will create 16 partitions so that we can compare it to the partitions we create from hashed keys in the next part of the assignment. The partitions are determined by the first letter of the composite key using the following partitions.

In [1]:
# partitions = (
#         ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
#         ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
#         ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
#         ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
#     )

In this case ('A', 'A') means the folder should contain all of the routes whose composite key starts with A. Similarly, ('E', 'F') should contain routes whose composite key starts with E or F.

The results/kv directory should contain the following folders.

In [2]:
# kv
# ├── kv_key=A
# ├── kv_key=B
# ├── kv_key=C-D
# ├── kv_key=E-F
# ├── kv_key=G-H
# ├── kv_key=I-J
# ├── kv_key=K-L
# ├── kv_key=M
# ├── kv_key=N
# ├── kv_key=O-P
# ├── kv_key=Q-R
# ├── kv_key=S-T
# ├── kv_key=U
# ├── kv_key=V
# ├── kv_key=W-X
# └── kv_key=Y-Z

An easy way to create this directory structure is to create a new key called kv_key from the key column and use the to_parquet method with partition_cols=['kv_key'] to save a partitioned dataset.

## Part A: Work

In [3]:
import pandas as pd
import json
import pygeohash
import math

In [4]:
df = pd.read_parquet('~/dsc650/dsc650/assignments/assignment03/results/routes.parquet')

In [5]:
df.head()

,airline,src_airport,dst_airport,codeshare,equipment
0,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2965.0, 'name': 'Sochi Internat...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2]
1,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2]
2,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2962.0, 'name': 'Mineralnyye Vo...",False,[CR2]
3,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2]
4,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 4078.0, 'name': 'Tolmachevo Air...",False,[CR2]


In [6]:
def make_key(row):
    try:
        # not sure if row.airline.get('iata') would be better
        airline = row.airline['iata']
        src_airport = row.src_airport['iata']
        dst_airport = row.dst_airport['iata']
        key = f'{src_airport}{dst_airport}{airline}'
    except:
        # Provide a default value, just in case
        key = 'ZZZZZZZZ'
    return(key)

In [7]:
df['key'] = df.apply(make_key, axis=1)

In [8]:
df.head()

,airline,src_airport,dst_airport,codeshare,equipment,key
0,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2965.0, 'name': 'Sochi Internat...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],AERKZN2B
1,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],ASFKZN2B
2,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2962.0, 'name': 'Mineralnyye Vo...",False,[CR2],ASFMRV2B
3,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],CEKKZN2B
4,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 4078.0, 'name': 'Tolmachevo Air...",False,[CR2],CEKOVB2B


In [9]:
def partition_keys(row):
    key_first_char = row.key[0]
    if key_first_char in ['C','D']:
        key_first_char = 'C-D'
    if key_first_char in ['E','F']:
        key_first_char = 'E-F'
    if key_first_char in ['G','H']:
        key_first_char = 'G-H'
    if key_first_char in ['I','J']:
        key_first_char = 'I-J'
    if key_first_char in ['K','L']:
        key_first_char = 'K-L'
    if key_first_char in ['O','P']:
        key_first_char = 'O-P'
    if key_first_char in ['Q','R']:
        key_first_char = 'Q-R'
    if key_first_char in ['S','T']:
        key_first_char = 'S-T'
    if key_first_char in ['W','X']:
        key_first_char = 'W-X'
    if key_first_char in ['Y','Z']:
        key_first_char = 'Y-Z'
    return(key_first_char)

In [10]:
df['kv_key'] = df.apply(partition_keys, axis=1)

In [11]:
df.head()

,airline,src_airport,dst_airport,codeshare,equipment,key,kv_key
0,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2965.0, 'name': 'Sochi Internat...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],AERKZN2B,A
1,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],ASFKZN2B,A
2,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2962.0, 'name': 'Mineralnyye Vo...",False,[CR2],ASFMRV2B,A
3,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],CEKKZN2B,C-D
4,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 4078.0, 'name': 'Tolmachevo Air...",False,[CR2],CEKOVB2B,C-D


In [12]:
# partition into 16 sections, regardless of size
df.to_parquet('./results/kv/',partition_cols=['kv_key'])

## Part B: Instructions

Next, we are going to partition the dataset again, but this time we will partition by the hash value of the key. The following is a function that will create a SHA256 hash of the input key and return a hexadecimal string representation of the hash.

In [13]:
# import hashlib

# def hash_key(key):
#     m = hashlib.sha256()
#     m.update(str(key).encode('utf-8'))
#     return m.hexdigest()

We will partition the data using the first character of the hexadecimal hash. As such, there are 16 possible partitions. Create a new column called hashed that is a hashed value of the key column. Next, create a partitioned dataset based on the first character of the hashed key and save the results to results/hash. The directory should contain the following folders.

In [14]:
# hash
# ├── hash_key=0
# ├── hash_key=1
# ├── hash_key=2
# ├── hash_key=3
# ├── hash_key=4
# ├── hash_key=5
# ├── hash_key=6
# ├── hash_key=7
# ├── hash_key=8
# ├── hash_key=9
# ├── hash_key=A
# ├── hash_key=B
# ├── hash_key=C
# ├── hash_key=D
# ├── hash_key=E

## Part B: Work

In [15]:
import hashlib

def hash_key(key):
    # returns a hashed version of the string passed in
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

def make_hash_key(row):
    # returns first character of hash of 'key' from row
    key = row.key
    return(hash_key(key)[0])

In [16]:
df['hash_key'] = df.apply(make_hash_key, axis=1)

In [17]:
df

,airline,src_airport,dst_airport,codeshare,equipment,key,kv_key,hash_key
0,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2965.0, 'name': 'Sochi Internat...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],AERKZN2B,A,6
1,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],ASFKZN2B,A,9
2,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2962.0, 'name': 'Mineralnyye Vo...",False,[CR2],ASFMRV2B,A,1
3,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],CEKKZN2B,C-D,3
4,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 4078.0, 'name': 'Tolmachevo Air...",False,[CR2],CEKOVB2B,C-D,1
...,...,...,...,...,...,...,...,...
67658,"{'airline_id': 4178, 'name': 'Regional Express...","{'airport_id': 6334.0, 'name': 'Whyalla Airpor...","{'airport_id': 3341.0, 'name': 'Adelaide Inter...",False,[SF3],WYAADLZL,W-X,f
67659,"{'airline_id': 19016, 'name': 'Apache Air', 'a...","{'airport_id': 4029.0, 'name': 'Domodedovo Int...","{'airport_id': 2912.0, 'name': 'Manas Internat...",False,[734],DMEFRUZM,C-D,8
67660,"{'airline_id': 19016, 'name': 'Apache Air', 'a...","{'airport_id': 2912.0, 'name': 'Manas Internat...","{'airport_id': 4029.0, 'name': 'Domodedovo Int...",False,[734],FRUDMEZM,E-F,e
67661,"{'airline_id': 19016, 'name': 'Apache Air', 'a...","{'airport_id': 2912.0, 'name': 'Manas Internat...","{'airport_id': 2913.0, 'name': 'Osh Airport', ...",False,[734],FRUOSSZM,E-F,8


In [18]:
# parition by first char of hash
df.to_parquet('./results/hash/',partition_cols=['hash_key'])

## Part C: Instructions

Finally, we will simulate multiple geographically distributed data centers. For this example, we will assume we have three data centers located in the western, central, and eastern United States. Google lists the locations of their data centers and we will use the following locations for our three data centers.

In [19]:
# West
# The Dalles, Oregon
# Latitude: 45.5945645
# Longitude: -121.1786823

# Central
# Papillion, NE
# Latitude: 41.1544433
# Longitude: -96.0422378

# East
# Loudoun County, Virginia
# Latitude: 39.08344
# Longitude: -77.6497145

Assume that you have an application that provides routes for each of the source airports and you want to store routes in the data center closest to the source airport. The output folders should look as follows.

In [20]:
# geo
# ├── location=central
# ├── location=east
# └── location=west

## Part C: Work

In [21]:
# Just some data to start with
datacenter_dict = [
    {
        "location": "west",
        "city": "The Dalles, Oregon",
        "latitude": 45.5945645,
        "longitude": -121.1786823
    },
    {
        "location": "central",
        "city": "Papillion, NE",
        "latitude": 41.1544433,
        "longitude": -96.0422378
    },
    {
        "location": "east",
        "city": "Loudoun County, Virginia",
        "latitude": 39.08344,
        "longitude": -77.6497145
    }
]


In [22]:
# Updated the json to include the geohash field
for datacenter in datacenter_dict:
    datacenter['geohash'] = pygeohash.encode(datacenter['latitude'], datacenter['longitude'])
    
datacenter_dict

[{'location': 'west',
  'city': 'The Dalles, Oregon',
  'latitude': 45.5945645,
  'longitude': -121.1786823,
  'geohash': 'c21g6s0rs4c7'},
 {'location': 'central',
  'city': 'Papillion, NE',
  'latitude': 41.1544433,
  'longitude': -96.0422378,
  'geohash': '9z7dnebnj8kb'},
 {'location': 'east',
  'city': 'Loudoun County, Virginia',
  'latitude': 39.08344,
  'longitude': -77.6497145,
  'geohash': 'dqby34cjw922'}]

In [23]:
def datacenter_search(latitude, longitude, datacenter_dict):
    # returns the 'location' of closest datacenter
    try:
        search_hash = pygeohash.encode(latitude, longitude)
        closest_datacenter_distance = math.inf
        for datacenter in datacenter_dict:
            dist = pygeohash.geohash_haversine_distance(search_hash, datacenter['geohash'])
            if dist < closest_datacenter_distance:
                closest_datacenter = datacenter
                closest_datacenter_distance = dist
        return(closest_datacenter['location'])
    except Exception as e:
        # Just in case of bad input, have a default value
        return('central')

def make_datacenter_key(row):
    try:
        # lets assume lat and long exist
        lat = row.src_airport['latitude']
        long = row.src_airport['longitude']
    except:
        # Oh, they don't?
        lat = None
        long = None
    datacenter_key = datacenter_search(lat,long,datacenter_dict)
    return(datacenter_key)

In [24]:
# A quick test
datacenter_search(41.1499988, -95.91779,datacenter_dict)

'central'

In [25]:
# check that it doesnt always return 'central'
datacenter_search(45.5945645,-121.1786823,datacenter_dict)

'west'

In [26]:
# make a new field
df['location'] = df.apply(make_datacenter_key, axis=1)

In [27]:
df.head(10)

,airline,src_airport,dst_airport,codeshare,equipment,key,kv_key,hash_key,location
0,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2965.0, 'name': 'Sochi Internat...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],AERKZN2B,A,6,east
1,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],ASFKZN2B,A,9,east
2,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2966.0, 'name': 'Astrakhan Airp...","{'airport_id': 2962.0, 'name': 'Mineralnyye Vo...",False,[CR2],ASFMRV2B,A,1,east
3,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],CEKKZN2B,C-D,3,west
4,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 2968.0, 'name': 'Chelyabinsk Ba...","{'airport_id': 4078.0, 'name': 'Tolmachevo Air...",False,[CR2],CEKOVB2B,C-D,1,west
5,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 4029.0, 'name': 'Domodedovo Int...","{'airport_id': 2990.0, 'name': 'Kazan Internat...",False,[CR2],DMEKZN2B,C-D,e,east
6,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 4029.0, 'name': 'Domodedovo Int...","{'airport_id': 6969.0, 'name': 'Begishevo Airp...",False,[CR2],DMENBC2B,C-D,3,east
7,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 4029.0, 'name': 'Domodedovo Int...",None,False,[CR2],ZZZZZZZZ,Y-Z,e,east
8,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 4029.0, 'name': 'Domodedovo Int...","{'airport_id': 6160.0, 'name': 'Bugulma Airpor...",False,[CR2],DMEUUA2B,C-D,f,east
9,"{'airline_id': 410, 'name': 'Aerocondor', 'ali...","{'airport_id': 6156.0, 'name': 'Belgorod Inter...","{'airport_id': 2952.0, 'name': 'Khrabrovo Airp...",False,[CR2],EGOKGD2B,E-F,a,east


In [28]:
#partition by location
df.to_parquet('./results/geo/',partition_cols=['location'])

## Part D: Instructions

Create a Python function that takes as input a list of keys and the number of partitions and returns a list of keys sorted into the specified number of partitions. The partitions should be roughly equal in size. Furthermore, the partitions should have the property that each partition contains all the keys between the least key in the partition and the greatest key in the partition. In other words, the partitions should be ordered.

In [29]:
# def balance_partitions(keys, num_partitions):
#     partitions = []
#     return partitions

## Part D: Work

In [30]:
def balance_partitions(keys, num_partitions):
    # sort the keys, if they aren't already
    keys.sort()
    # partition_size should be about len(keys) / num_partitions
    partition_size, remainder = divmod(len(keys), num_partitions)    
    partitions = []
    for i in range(num_partitions):
        start_index = i * partition_size + min(i, remainder)
        finish_index = (i + 1) * partition_size + min(i + 1, remainder)
        partition = keys[start_index:finish_index]
        partitions.append(partition)
    return(partitions)

keylist = [9,1,3,2,4,6,5,7,8]

for i in range(1,len(keylist)+1):
    print('--------------|-----------------------------------------------')
    print(f'{i} partitions: | {balance_partitions(keylist,i)}')

--------------|-----------------------------------------------
1 partitions: | [[1, 2, 3, 4, 5, 6, 7, 8, 9]]
--------------|-----------------------------------------------
2 partitions: | [[1, 2, 3, 4, 5], [6, 7, 8, 9]]
--------------|-----------------------------------------------
3 partitions: | [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
--------------|-----------------------------------------------
4 partitions: | [[1, 2, 3], [4, 5], [6, 7], [8, 9]]
--------------|-----------------------------------------------
5 partitions: | [[1, 2], [3, 4], [5, 6], [7, 8], [9]]
--------------|-----------------------------------------------
6 partitions: | [[1, 2], [3, 4], [5, 6], [7], [8], [9]]
--------------|-----------------------------------------------
7 partitions: | [[1, 2], [3, 4], [5], [6], [7], [8], [9]]
--------------|-----------------------------------------------
8 partitions: | [[1, 2], [3], [4], [5], [6], [7], [8], [9]]
--------------|-----------------------------------------------
9 partiti